<h2>Imports</h2>

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
import copy

<h2>Utils

In [ ]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()


def get_nth_maximum(arr,n):
    uniqueValues = list(arr.flatten())
    uniqueValues.sort()
    if len(uniqueValues) < n:
        return uniqueValues[0]
    return uniqueValues[len(uniqueValues)-n]
    # sortedmatrix.sort()
    # # print(arr.shape)
    # if sortedmatrix.size == 0:
    #     return -1
    # return sortedmatrix[0 if n > sortedmatrix.size else -n] 

<h2>Constants</h2>

In [ ]:
datasetPath = "Data/dinoSparseRing/*.png"
ß1 = 2
ß2 = 32  
# We associate with p a reference image R(p),the images S(p) where p should be visible and the images T(p) where it is truly found  
patchModel = {
    "R":None,
    "S":set,
    "T":set
}
# The cell C(i, j) keeps track of two different sets Qt(i, j) and Qf(i, j)
cell = {
    "Qt":set,
    "Qf":set
}
# We associate with each image I a regular grid of β1×β1 pixel^2 cells
imageModel = {
    "img":np.ndarray,
    "grid":np.ndarray
}

<h2>Get Input

In [ ]:
# Initialize image model from a given path
def init_imgs(datasetPath):
    # Read imgs
    imgs = [cv.rotate(cv.imread(file),cv.ROTATE_90_COUNTERCLOCKWISE) for file in glob(datasetPath)]

    # Construct corresponding image grid
    grids = [np.array([np.array([cell for x in range(0,img.shape[1]//ß1)]) for y in range(0,img.shape[0]//ß1)]) for img in imgs]

    return imgs,grids

<h2>Feature Detection

In [ ]:
# Get Harris and DoG operators for a given image
def get_dog_harris(img):
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)
            
    # Get DoG
    g1 = cv.GaussianBlur(gray,(0,0),sigmaX=1)
    g2 = cv.GaussianBlur(gray,(0,0),sigmaX=1*np.sqrt(2))
    diff = cv.absdiff(g1,g2)
    dog = diff * gray

    # Get Harris
    bSize = 3
    kSize = 1
    corners = cv.cornerHarris(src=gray,blockSize=bSize,ksize=kSize,k=0.06)
    # corners = cv.dilate(corners,None)
    
    return dog , corners


def sparse_dog_harris(dog,harris):
    n = 4
    sparseDog = copy.copy(dog)
    sparseHarris = copy.copy(harris)
    sparseDogPositions = []
    sparseHarrisPositions = []
    for yIdx in range(0,len(dog),ß2):
        for xIdx in range(0,len(dog[0]),ß2):
            nThMaximumDog = get_nth_maximum(dog[yIdx:yIdx+ß2,xIdx:xIdx+ß2],n)
            if nThMaximumDog != -1:
                found = False
                for rowIdx,row in enumerate(dog[yIdx:yIdx+ß2]):
                    for columnIdx,column in enumerate(row[xIdx:xIdx+ß2]):
                        if not found and column == nThMaximumDog:
                            found = True
                            if column != 0:
                                sparseDogPositions.append((yIdx+rowIdx,xIdx+columnIdx))
                        else:
                            sparseDog[yIdx+rowIdx,xIdx+columnIdx] = 0
                # sparseDog[yIdx:yIdx+ß2,xIdx:xIdx+ß2] = sparseDog[yIdx:yIdx+ß2,xIdx:xIdx+ß2]*(sparseDog[yIdx:yIdx+ß2,xIdx:xIdx+ß2] == nThMaximumDog)
            nThMaximumHarris = get_nth_maximum(harris[yIdx:yIdx+ß2,xIdx:xIdx+ß2],n)
            if nThMaximumHarris != -1:
                found = False
                for rowIdx,row in enumerate(harris[yIdx:yIdx+ß2]):
                    for columnIdx,column in enumerate(row[xIdx:xIdx+ß2]):
                        if not found and column == nThMaximumHarris:
                            found = True
                            if column != 0:
                                sparseHarrisPositions.append((yIdx+rowIdx,xIdx+columnIdx))
                        else:
                            sparseHarris[yIdx+rowIdx,xIdx+columnIdx] = 0
                # sparseHarris[yIdx:yIdx+ß2,xIdx:xIdx+ß2] = sparseHarris[yIdx:yIdx+ß2,xIdx:xIdx+ß2]*(sparseHarris[yIdx:yIdx+ß2,xIdx:xIdx+ß2] == nThMaximumHarris)
            # show_images([dog[yIdx:yIdx+ß2,xIdx:xIdx+ß2],sparseDog[yIdx:yIdx+ß2,xIdx:xIdx+ß2],harris[yIdx:yIdx+ß2,xIdx:xIdx+ß2],sparseHarris[yIdx:yIdx+ß2,xIdx:xIdx+ß2]],['before dog','after dog','before harris','after harris'])

    # sparseDog = cv.dilate(sparseDog,None)
    # sparseDog = cv.dilate(sparseDog,None)
    # sparseHarris = cv.dilate(sparseHarris,None)
    # sparseHarris = cv.dilate(sparseHarris,None)
    return sparseDog,sparseHarris,sparseDogPositions,sparseHarrisPositions


<h2>Get Fundmental Matrix using SIFT

In [ ]:
# Get the fundmental matrix between 2 pictures
def getFundmentalMatrix(idx1,idx2):
    sift = cv.xfeatures2d.SIFT_create()
    # find keypoints and descriptors with SIFT
    kp1,des1 = sift.detectAndCompute(images[idx1],None)
    kp2,des2 = sift.detectAndCompute(images[idx2],None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)

    pts1 = []
    pts2 = []

    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
            
    pts1 = np.float32(pts1)
    pts2 = np.float32(pts2)
    fundmentalMat, _ = cv.findFundamentalMat(pts1,pts2,cv.FM_LMEDS)
    print(("Fundmental Matrix between image[%d] and image[%d]:\n%a") % (idx1,idx2,fundmentalMat))
    return fundmentalMat

<h2>Draw Epilines

In [ ]:
# Draw the epilines corresponding to a point in the first image
# Draw also the points satisfying epipolar consistancy 
def drawlines(img1,lines,pts1):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''

    reducedFeaturesImage = copy.copy(img1)
    fullFeaturesImage = copy.copy(img1)
    r,c,_ = reducedFeaturesImage.shape
    
    for r,pt1 in zip(lines,pts1):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        cv.line(reducedFeaturesImage, (x0,y0), (x1,y1), color,1)
        cv.line(fullFeaturesImage, (x0,y0), (x1,y1), color,1)

    a = lines[0][0]
    b = lines[0][1]
    c = lines[0][2]

    for pt in pts1:
        # color = tuple(np.random.randint(0,255,3).tolist())
        ptx = pt[0]
        pty = pt[1]
        if abs(a*ptx+b*pty+c) <= 2 :
            cv.circle(reducedFeaturesImage,tuple(pt),5,(0,255,0),-1)
            cv.circle(fullFeaturesImage,tuple(pt),5,(0,255,0),-1)
        else:
            cv.circle(fullFeaturesImage,tuple(pt),5,(255,0,0),-1)

    return fullFeaturesImage,reducedFeaturesImage

<h2>Main

In [ ]:
images,grids = init_imgs(datasetPath)
imagesFeatures = []
for idx,image in enumerate(images):
    dog,harris = get_dog_harris(image)
    sparseDog,sparseHarris,dogPositions,harrisPositions = sparse_dog_harris(dog,harris)
    imagesFeatures.append(
        {
            "image":images[idx],
            "grid":grids[idx],
            "dog":dog,
            "harris":harris,
            "sparseDog":sparseDog,
            "sparseHarris":sparseHarris,
            "dogPositions":dogPositions,
            "harrisPositions":harrisPositions
        }
    )

show_images([imagesFeatures[0]["dog"],imagesFeatures[0]["sparseDog"],imagesFeatures[0]["harris"],imagesFeatures[0]["sparseHarris"]],['dog','sparse dog','harris','sparse harris'])

In [ ]:
for i in range(1,len(images)):
    fundmentalMat = getFundmentalMatrix(0,i)

    pts1 = np.int32([imagesFeatures[0]["dogPositions"][0]])
    pts2 = np.int32(imagesFeatures[i]["dogPositions"])
    # Get the epilines of features in left image on the right image
    # parameter1: points required to get its epilines in the other image
    # parameter2: which image that points are belong, 1-left 2-right
    # parameter3: fundmental matrix between the 2 images
    # returns list of epilines that lie on the other image and corresponding to the points
    lines = cv.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,fundmentalMat)
    lines = lines.reshape(-1,3)
    
    # draw the epiline on the other image
    # parameter1: the second image
    # parameter2: the epilines that lie on the second image
    # parameter3: the features lie on the second image
    fullFeaturesImage,reducedFeaturesImage = drawlines(images[i],lines,pts2)
    show_images([images[0],images[i],fullFeaturesImage,reducedFeaturesImage],["image"+str(0),"image"+str(i),"fullfeatures in image"+str(i),"reducedfeatures in image"+str(i)])